# 正則化と強化学習と最適化

参考：
* [A NOTE ON OPTIMIZATION FORMULATIONS OF MARKOV DECISION PROCESSES](https://arxiv.org/abs/2012.09417)

多くのMDP用のアルゴリズムはベルマン方程式における不動点を利用したものがほとんどです．
TabularやLinearの場合にはこうした手法は良く機能しますが，関数近似が入ると破綻することがあります（[RL_General_fitted_Q_iteration.ipynb](RL_General_fitted_Q_iteration.ipynb)など）

一方で，最適化問題として考えるとうまく行くことがあります（[RL_Convex_Fenchel_Duality_and_DICE.ipynb](RL_Convex_Fenchel_Duality_and_DICE.ipynb)など参照）．今回は特に正則化された強化学習問題と最適化としての定式化について見てみましょう．


表記：
* だいたいいつものMDPの表記と同じです．$P^a_{st}$は$s$と$a$によって状態$t$に至る確率とします．
* エントロピー：$h(\rho):=\sum_{a \in \mathcal{A}} \rho^a \log \frac{\rho^a}{\sum_{b \in \mathcal{A}} \rho^b}$
    * ここで$\rho \in \mathbb{R}^\mathcal{A}$はNon-negativeな値です．

## 凸最適化問題としての定式化

[RL_Convex_as_LP.ipynb](RL_Convex_as_LP.ipynb)を思い出すと，正則化がない場合のMDPは次の線型計画法で解くことができます．
$$
\min _{v_s} \max _{\mu_s^a \geq 0} \sum_{s \in \mathcal{S}} e_s v_s+\sum_{s, a}\left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t-v_s\right) \mu_s^a
$$
ここで$\mu_s^a$はoccupancy measureに対応するわけですね．
正則化がある場合はどうでしょうか．
証明は後で行うとして，次のPrimal-dual問題について考えてみましょう．

---

**Primal-dual問題**

$$
\min _{v_s} \sup _{\mu_s^a>0} \sum_{s \in \mathcal{S}} e_s v_s+\sum_{s, a}\left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t-v_s\right) \mu_s^a-\sum_{s \in \mathcal{S}} h\left(\mu_s\right)
$$

$h(\mu_s)$は$\mu_s$について凸なので，この問題の目的関数は$\mu_s^a$について凹です．
これを解くと，実は正則化された最適価値が求まります（後で示します）．

---

**Primal問題**

次の表記を導入しましょう．
$$
\mu_s^a=w_s \pi_s^a \text { with } w_s=\sum_{a \in \mathcal{A}} \mu_s^a \text { and } \pi \in \Delta^{|\mathcal{S}|}
$$
つまり，Occupancy measureを方策と状態上の確率に分割しているだけですね．このとき，
$$
\min _{v_s} \sup _{\pi \in \Delta|\mathcal{S}|, \pi_s^a, w_s>0} \sum_{s \in \mathcal{S}} e_s v_s+\sum_{s, a}\left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t-v_s\right) \pi_s^a w_s-\sum_{s \in \mathcal{S}} w_s h\left(\pi_s\right)
$$
として上の問題を書き直すことができます．これは更に次と等価です：

$$
\min _{v_s}\left(\sum_{s \in \mathcal{S}} e_s v_s+\sup _{w_s>0} \sum_{s \in \mathcal{S}} w_s \cdot \max _{\pi_s \in \Delta}\left(\sum_{a \in \mathcal{A}}\left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t-v_s\right) \pi_s^a-h\left(\pi_s\right)\right)\right)
$$

これを改めて制約付き最適化問題に戻すと，上の問題は次と等価です：
$$
\min _{v_s} e^{\top} v \text {, s.t. } \forall s, \max _{\pi_s \in \Delta} \sum_{a \in \mathcal{A}}\left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t-v_s\right) \pi_s^a-h\left(\pi_s\right) \leq 0 \text {, }
$$

ベクトル形式で単純化すると，
$$
\min _v e^{\top} v \text {, s.t. } \max _\pi r^\pi+\gamma P^\pi v-h^\pi \leq v \text {. }
$$

と等価ですね．ところで，Fenchel共役の性質を使えば，制約の中身の$\max_\pi$は極大値が$\pi_s^a \propto \exp \left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t-v_s\right)$で与えられます．これを使うと，制約の部分は
$$
\sum_{a \in \mathcal{A}} \exp \left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t\right) \leq e^{v_s} .
$$

と同じです．両辺にlogを取ると，最適化問題は
$$
\min _{v_s} e^{\top} v \text {, s.t. } \forall s, v_s \geq \log \left(\sum_{a \in \mathcal{A}} \exp \left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t\right)\right)
$$

と等価であることがわかります．

---

**Dual問題**

SionのMinimax定理から，今までの最適化問題は
$$
\sup _{\mu_s^a>0} \min _{v_s} \sum_{s \in \mathcal{S}} e_s v_s+\sum_{s, a}\left(r_s+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t-v_s\right) \mu_s^a-\sum_{s \in \mathcal{S}} h\left(\mu_s\right) .
$$
と等価です．$v$について微分すると，
$$
\sum_{a \in \mathcal{A}}\left(I-\gamma\left(P^a\right)^{\top}\right) \mu^a=e
$$
が条件として与えられるので，結局Primal-dual問題は
$$
\sup _{\mu^a>0} \sum_{a \in \mathcal{A}}\left(r^a\right)^{\top} \mu^a-\sum_{s \in \mathcal{S}} h\left(\mu_s\right) \text {, s.t. } \sum_{a \in \mathcal{A}}\left(I-\gamma\left(P^a\right)^{\top}\right) \mu^a=e \text {. }
$$
と等価であることがわかります．


## 凸最適化問題とRLの等価性について

**双対問題と方策勾配**

上で見た双対問題を，$\mu_s^a=w_s \pi_s^a$として変形すると，その制約の部分は
$$
\forall s, \sum_{a \in \mathcal{A}} \pi_s^a w_s-\gamma \sum_{a, t} P_{t s}^a \pi_t^a w_t=e_s, \quad \text { or } \quad w=\left(I-\gamma\left(P^\pi\right)^{\top}\right)^{-1} e
$$
と等価です．$w$についての解は$\pi$に依存しているので，$w^\pi$として表記することにすれば，目的関数の部分は
$$
\max _{\pi \in \Delta|\mathcal{S}|} r^\pi \cdot w^\pi-\sum_{s \in \mathcal{S}} w_s^\pi\left(\sum_{a \in \mathcal{A}} \pi_s^a \log \pi_s^a\right)
$$
と同じです．また，$h^\pi \in \mathbb{R}^\mathcal{S}$なる表記を使えば，
$$
\max _{\pi \in \Delta^{|\mathcal{S}|}}\left(r^\pi-h^\pi\right) \cdot w^\pi, \quad \text { or } \quad \max _{\pi \in \Delta^{|\mathcal{S}|}} e^{\top}\left(I-\gamma P^\pi\right)^{-1}\left(r^\pi-h^\pi\right) \text {. }
$$
として書くことができます．まとめると，Dual問題は次と同じです：

$$
\max _{\pi \in \Delta|\mathcal{S}|} e^{\top} v^\pi, \quad \text { s.t., } \quad v^\pi=r^\pi-h^\pi+\gamma P^\pi v^\pi \text {, }
$$

これは明らかに方策勾配法による最適化と同じですね．

---

**Primal問題とBellman方程式の等価性**

まず，Regularized Bellman 方程式は
$$
v=\max _{\pi \in \Delta|\mathcal{S}|} r^\pi+\gamma P^\pi v-h^\pi .
$$

であり，各状態は
$$\left.v_s=\max _{\pi_s \in \Delta} \sum_{a \in \mathcal{A}}\left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t\right)\right) \pi_s^a-h\left(\pi_s\right)$$
に対応します．すると価値関数は
$$v_s=\log \left(\sum_{a \in \mathcal{A}} \exp \left(r_s^a+\gamma \sum_{t \in \mathcal{S}} P_{s t}^a v_t\right)\right)$$
の形式になります．

**Bellman方程式→Primal問題**


**Primal問題→Bellman方程式**

この２つは論文を参照すればすぐわかるので省略します．
[A NOTE ON OPTIMIZATION FORMULATIONS OF MARKOV DECISION PROCESSES](https://arxiv.org/abs/2012.09417)参照．


# Primal-dual問題を高速に解く

改めてPrimal-dual問題を見てみましょう．ちょっと表記を変えます($K_a=I-\gamma P_a$ and $K_\pi=I-\gamma P_\pi$としました)：

$$
\min _v \max _u E_0(v, u):=\sum_s e_s v_s+\sum_{s a} u_{s a}\left(r_{s a}-\left(K_a v\right)_s\right)-\tau \sum_{s a} u_{s a} \log \frac{u_{s a}}{\tilde{u}_s} .
$$

これをprimal-dual形式で解いてもいいんですが，$v$について線形であり，First order最適化を使うと収束が基本的に遅いです（そうなのか）．
そこで，次のような二次形式の問題を考えましょう．

$$
\min _v \max _u E(v, u):=\frac{\alpha}{2} \sum_s v_s^2+\sum_{s a} u_{s a}\left(r_{s a}-\left(K_a v\right)_s\right)-\tau \sum_{s a} u_{s a} \log \frac{u_{s a}}{\tilde{u}_s}
$$

実はこの問題を考えても$v^*$は同じです（下の図のイメージ）．また，二乗ではなく任意の強凸関数で置き換えても解は同じです．
証明はTheorem 2.1を参照してください．

![quadratic](figs/regularized_quadratic_formulation.png)

## 自然勾配による解法

$E$について勾配を取ると，
$$
\begin{aligned}
\frac{\partial E}{\partial v_{s^{\prime}}} & =\alpha v_{s^{\prime}}-\sum_{s a} K_{a s s^{\prime}} u_{s a}, \quad s^{\prime} \in S, \\
\frac{\partial E}{\partial u_{s a}} & =\left(r_{s a}-\sum_{s^{\prime}} K_{a s s^{\prime}} v_{s^{\prime}}\right)-\tau \log \frac{u_{s a}}{\tilde{u}_s}, \quad(s, a) \in S \times A
\end{aligned}
$$

が得られます．また，二次微分は
$$
\begin{aligned}
\frac{\partial^2 E}{\partial v_s \partial v_{s^{\prime}}} & =\alpha \delta_{s s^{\prime}}, \quad\left(s, s^{\prime}\right) \in S \times S \\
\frac{\partial^2 E}{\partial u_{s a} \partial u_{s^{\prime} a^{\prime}}} & =-\tau \delta_{s s^{\prime}}\left(\frac{\delta_{a a^{\prime}}}{u_{s a}}-\frac{1}{\tilde{u}_s}\right), \quad\left(s, s^{\prime}, a, a^{\prime}\right) \in S^2 \times A^2 .
\end{aligned}
$$
です．